#### Solutions to the problems of Chapter 9
### Priority Queues and Heaps

### helper classes 

In [14]:
class Empty(Exception):
    pass 
class _DoublyLinkedBase:
    class _Node:
        __slots__='_element', '_prev', '_next'
        
        def __init__(self, element, prev, nxt):
            self._element = element 
            self._prev = prev
            self._next = nxt 
            
    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer 
        self._trailer._prev = self._header
        self._size = 0 
        
    def __len__(self): 
        return self._size 
    
    def is_empty(self):
        return self._size == 0 
    
    def _insert_between(self, e, predecessor, successor):
        newest = self._Node(e, predecessor, successor)
        predecessor._next = newest
        successor._prev = newest 
        self._size += 1
        return newest 
    
    def _delete_node(self, node):
        predecessor = node._prev 
        successor = node._next 
        predecessor._next = successor 
        successor._prev = predecessor 
        self._size -= 1
        element = node._element 
        node._prev = node._next = node._element = None
        return element 

class PositionalList(_DoublyLinkedBase):
    def __init__(self):
        super().__init__() 
        
    class Position: 
        def __init__(self, container, node):
            self._container = container 
            self._node = node 
            
        def element(self):
            return self._node._element 
        def __eq__(self, other): 
            return type(other) is type(self) and other._node is self._node
        def __ne__(self, other):
            return not(self==other)
    
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper position type')
        if p._container is not self:
            raise ValueError('p does not belong to this type')
        if p._node._next is None:
            raise ValueError('p is no longer valid')
        return p._node 
    
    def _make_position(self, node):
        if (node is self._header) or (node is self._trailer):
            return None
        else:
            return self.Position(self, node)
        
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        cursor = self.first() 
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
            
    def _insert_between(self, e, predecessor, successor):
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)
    
    def add_after(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original, original._next)
    
    def delete(self, p):
        original = self._validate(p)
        return self._delete_node(original)
    
    def replace(self, p, e):
        original = self._validate(p)
        old_value = original._element 
        original._element = e
        return old_value

#### Implementation to the Priority Queue ADT,  UnsortedPriorityQueue   

In [15]:
class PriorityQueueBase:
    class _Item:
        __slots__= '_key', '_value'
        def __init__(self, key, value):
            self._key = key
            self._value = value 
        def __lt__(self, other):
            return self._key < other._key 
    def is_empty(self):
        return len(self) ==0 

In [16]:
class UnsortedPriorityQueue(PriorityQueueBase): 
    def __init__(self):
        self._data = PositionalList() 
    
    def __len__(self):
        return len(self._data)
    
    def add(self, key, value):
        self._data.add_last(self._Item(key, value))
    
    def _find_min(self):
        if self.is_empty():
            raise Empty("Empty Prioirty Queue")
        minimum = self._data.first() 
        current = self._data.after(minimum)
        while current is not None:
            if current.element()< minimum.element():
                minimum = current
            current = self._data.after(current)
        return minimum 
    
    def min(self):
        item = self._find_min().element() 
        return (item._key, item._value) 
    
    def remove_min(self):
        minimum = self._find_min()
        item = self._data.delete(minimum) 
        return (item._key, item._value) 

In [17]:
class SortedPriorityQueue(PriorityQueueBase):
    def __init__(self):
        self._data = PositionalList() 
    def __len__(self):
        return len(self._data)
    def add(self, key, value):
        new = self._Item(key, value)
        current = self._data.last() 
        while current is not None and new<current.element(): 
            current = self._data.before(current)
        if current is None: 
            self._data.add_first(new)
        else:
            self._data.add_after(current, new)
            
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.first().element()
        return (min_item._key, min_item._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.delete(self._data.first())
        return (min_item._key, min_item._value)

#### Heap Data structure 

In [32]:
class HeapPriorityQueue(PriorityQueueBase):
    def __init__(self, contents = ()): 
        self._data = [self._Item(k, v) for k, v in contents]
        if len(self._data)>1:
            self._heapify() 
            
    def __len__(self):
        return len(self._data) 
    
    def _heapify(self): 
        start = self.parent(len(self._data)-1)  
        for j in range(start, -1, -1):
            self._downheap(j)
    
    def add(self, key, value):
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data)-1)
    
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue!")
        mini = self._data[0]
        return (mini._key, mini._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue!")
        self._swap(0, len(self._data)-1)
        mini = self._data.pop()
        self._downheap(0)
        return (mini._key, mini._value) 
    
    def _parent(self, j):
        return (j-1)//2 
    
    def _left(self, j):
        return (2*j) + 1 
    
    def _right(self, j):
        return (2*j) + 2 
    
    def _has_left(self, j):
        return self._left(j) < len(self._data)
    
    def _has_right(self, j):
        return self._right(j) < len(self._data)
    
    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]
    
    def _upheap(self, j):
        parent = self._parent(j)
        if j>0 and (self._data[j]<self._data[parent]): 
            self._swap(j, parent)
            self._upheap(parent)
    
    def _downheap(self, j):
        if self._has_left(j):
            small_child = self._left(j)
            if self._has_right(j):
                right_child = self._right(j)
                if self._data[right_child]<self._data[small_child]:
                    small_child = right_child
            
            if self._data[small_child]<self._data[j]: 
                self._swap(j, small_child)
                self._downheap(small_child)

In [33]:
class AdaptableHeapPriorityQueue(HeapPriorityQueue):
    class Locator(HeapPriorityQueue._Item):
        __slot__ = '_index'
        def __init__(self, k, v, j):
            super().__init__(k, v)
            self._index = j
    
    def _swap(self, i, j):
        super()._swap(i, j)
        self._data[i]._index = i
        self._data[j]._index = j
        
    def _bubble(self, j):
        if j>0 and self._data[j] < self._data[self._parent(j)]:
            self._upheap(j)
        else:
            self._downheap(j)
            
    def add(self, key, value):
        token = self.Locator(key, value, len(self._data))
        self._data.append(token)
        self._upheap(len(self._data)-1)
        return token
    
    def update(self, loc, newkey, newval):
        j = loc._index 
        if not (0 <= j <len(self) and self._data[j] is loc):
            raise ValueError("InValid locator")
        loc._key = newkey
        loc._value = newval 
        self._bubble(j)
        
    def remove(self, loc):
        j = loc._index 
        if not(0<=j<len(self) and self._data[j]is loc):
            raise ValueError("Invalid locator")
        if j==len(self)-1:
            self._pop()
        else:
            self._swap(j, len(self)-1)
            self._data.pop()
            self._bubble(j)
        return(loc._key, loc._value)
    

***Reinforcement Problems***

R-9.5 The min method for the UnsortedPriorityQueue class executes in O(n)time, as analyzed in Table 9.2. Give a simple modification to the class so that min runs in O(1) time. Explain any necessary modifications to other methods of the class.

this can be done if we have a variable that keeps track of the min value with each addition. It checks if the newly added value is less than or equal to the current min. if less then it stores it. This variable can be accessed in O(1) time. 

R-9.6 Can you adapt your solution to the previous problem to make remove_min run in O(1) time for the UnsortedPriorityQueue class? Explain your answer.

I don't think so because we need to find the min each time we do a removal. 

R-9.12 Consider a situation in which a user has numeric keys and wishes to have a priority queue that is maximum-oriented. How could a standard (minoriented) priority queue be used for such a purpose?

replace each key with the negative value of that key. 

***Creativity Problems*** 

C-9.26 Show how to implement the stack ADT using only a priority queue and one additional integer instance variable.

In [38]:
class PQStack:
    counter = 0 
    def __init__(self):
        self._pq = HeapPriorityQueue()
        self._count = 0 
        
    def push(self, e): 
        self._count -= 1 
        self._pq.add(self._count, e)
        
    def pop(self):
        return self._pq.remove_min()[1]
    
    def top(self):
        return self._pq.min()[1]

In [39]:
stack = PQStack() 
stack.push("a")
stack.push("b")
stack.push("c")
stack.push("d")

print(stack.top())
print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())

d
d
c
b
a


C-9.27 Show how to implement the FIFO queue ADT using only a priority queue and one additional integer instance variable.

In [41]:
class PQFIFO:
    counter = 0 
    def __init__(self):
        self._pq = HeapPriorityQueue()
        self._count = 0 
        
    def enqueue(self, e): 
        self._count += 1 
        self._pq.add(self._count, e)
        
    def dequeue(self):
        return self._pq.remove_min()[1]
    
    def top(self):
        return self._pq.min()[1]

In [43]:
queue = PQFIFO() 
queue.enqueue("a")
queue.enqueue("b")
queue.enqueue("c")
queue.enqueue("d")

print(queue.top())
print(queue.dequeue())
print(queue.dequeue())

a
a
b


C-9.28 Professor Idle suggests the following solution to the previous problem.Whenever an item is inserted into the queue, it is assigned a key that is equal to the current size of the queue. Does such a strategy result in FIFO semantics? Prove that it is so or provide a counterexample.

no, think about the case when we do at least two deletions, the newely added element will have the same key as an exisiting element, which will cause a problem since we will not know where to put this one. 

C-9.29 Reimplement the SortedPriorityQueue using a Python list. Make sure to maintain remove min’s O(1) performance.

In [102]:
class SortedPQ(PriorityQueueBase): 
    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def add(self, key, value):
        new = self._Item(key, value)
        self._data.append(new)
        idx = len(self._data)-1
        while idx >= 0 and self._data[idx]._key>self._data[idx-1]._key:
            self._data[idx], self._data[idx-1] = self._data[idx-1], self._data[idx]
            idx -= 1 
            
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data[-1]
        return (min_item._key, min_item._value)

    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.pop(-1)
        return (min_item._key, min_item._value)
    
    def __str__(self):
        return''.join("("+str(item._key)+","+str(item._value)+")" for item in self._data) 

In [104]:
pq = SortedPQ()
pq.add(5, "a")
pq.add(2, "c")
pq.add(4, "d")
pq.add(1, "e")
print(pq)
print(pq.min())
print(pq.remove_min())
print(pq.min())
print(pq.remove_min())
print(pq.min())

(5,a)(4,d)(2,c)(1,e)
(1, 'e')
(1, 'e')
(2, 'c')
(2, 'c')
(4, 'd')


* C-9.30 Give a nonrecursive implementation of the upheap method for the class
HeapPriorityQueue.
* C-9.31 Give a nonrecursive implementation of the downheap method for the class HeapPriorityQueue.

#### Full non-recursive heap data structure 
##### We'll implement a max-heap because we already have a min-heap 

In [180]:
class HeapPQ(PriorityQueueBase):
    def __init__(self, contents = ()):
        self._data = [self._Item(k, v) for k, v in contents]
            
    def __len__(self):
        return len(self._data) 
    
    def _heapify(self):
        leafs_start = self._parent(len(self._data)-1)
        for i in range(leafs_start, -1, -1):
            self._downheap(i) 
    
    def add(self, key, value):
        self._data.append(self_Item(key, value))
        self._upheap(len(self._data)-1) 
    
    def min(self):
        if len(self._data) == 0:
            raise Empty('Prioirty Queue is Empty!')
        mini = self._data[0]
        return (mini._key, mini._value)
    
    def remove_min(self):
        if len(self._data) == 0:
            raise Empty('Prioiry Queue is Empty!')
        self._swap(0, len(self._data)-1)
        mini = self._data.pop()
        self._downheap(0)
        return (mini._key, mini._value) 
    
    def _parent(self, j):
        return (j-1)//2 
    
    def _left(self, j):
        return 2*j+1 
    
    def _right(self, j):
        return 2*j+2 
    
    def _has_left(self, j):
        return self._left(j)<len(self._data) 
    
    def _has_right(self, j):
        return self._right(j)<len(self._data) 
    
    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i] 
    
    def _upheap(self, j):
        while j>0 and self._data[self._parent(j)]._key < self._data[j]._key:
            self._swap(j, self._parent(j))
            j = self._parent(j) 
    
    def _downheap(self, j):
        while j<len(self._data) and self._has_left(j):
            larger = self._left(j)
            if self._has_right(j): 
                right = self._right(j)
                if self._data[right]._key>self._data[larger]._key:
                    larger = right
            if self._data[j] < self._data[larger]: 
                self._swap(j, larger)
            j = larger 
    
    def __repr__(self): 
        return ''.join("("+str(item._key)+","+str(item._value)+")" for item in self._data) 

In [183]:
pq = HeapPQ([(1, "a"), (2, "c"), (7,"b"), (4,"e"), (9, 'f'), (8, 'q')])
#pq._upheap(1)
print(pq)
#pq._downheap(0)
print(pq)
pq._heapify() 
print(pq)

(1,a)(2,c)(7,b)(4,e)(9,f)(8,q)
(1,a)(2,c)(7,b)(4,e)(9,f)(8,q)
(9,f)(4,e)(8,q)(1,a)(2,c)(7,b)


In [174]:
#(9,f)(4,e)(8,q)(1,a)(2,c)(7,b)

C-9.32 Assume that we are using a linked representation of a complete binary tree T , and an extra reference to the last node of that tree. Show how to update the reference to the last node after operations add or remove min in O(logn) time, where n is the current number of nodes of T . Be sure and handle all possible cases, as illustrated in Figure 9.12.

C-9.33 When using a linked-tree representation for a heap, an alternative method for finding the last node during an insertion in a heap T is to store, in the last node and each leaf node of T , a reference to the leaf node immediately to its right (wrapping to the first node in the next lower level for the rightmost leaf node). Show how to maintain such references in O(1) time per operation of the priority queue ADT assuming that T is implemented with a linked structure.